In [1]:
import pandas as pd
import numpy as np
import pprint
import os
import datetime

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

## Research Question
Can we predict game outcomes using all past seasons' team statistics?
For instance, if teams A and B are playing, can we use all the past seasons' statistics for team A and B to predict who will win?

#### Loading Each Dataset

In [2]:
# loading each dataset into a dictionary
datasets = {}
for file_name in os.listdir('archive'):
    if file_name.endswith('.csv'):
        datasets[file_name.split('.')[0]] = pd.read_csv('archive/' + file_name, low_memory=False)

list(datasets.keys())

['teams', 'players', 'games', 'ranking', 'games_details']

#### Cleaning and Organizing


In [3]:
# only keeping the team id, abbreviation, nickname, city, and arena capacity for each team in the teams dataset
# replacing NaN and 0 values with the mean
datasets['teams'] = datasets['teams'][['TEAM_ID', 'ABBREVIATION', 'NICKNAME', 'CITY', 'ARENACAPACITY']].fillna(0).replace(0, datasets['teams']["ARENACAPACITY"].mean().round(0))
datasets['teams'].head()

,TEAM_ID,ABBREVIATION,NICKNAME,CITY,ARENACAPACITY
0,1610612737,ATL,Hawks,Atlanta,18729.0
1,1610612738,BOS,Celtics,Boston,18624.0
2,1610612740,NOP,Pelicans,New Orleans,18553.0
3,1610612741,CHI,Bulls,Chicago,21711.0
4,1610612742,DAL,Mavericks,Dallas,19200.0


In [4]:
datasets['players'].sample(5)

,PLAYER_NAME,TEAM_ID,PLAYER_ID,SEASON
5415,Jermaine Taylor,1610612750,201966,2012
624,Hassani Gravett,1610612753,1629755,2019
420,Tyler Cook,1610612739,1629076,2019
7142,Stephen Jackson,1610612744,1536,2009
2104,Iman Shumpert,1610612739,202697,2016


In [5]:
datasets['game_details'] = datasets['games_details'][["GAME_ID", "TEAM_ID", "MIN", "FG_PCT", "FG3_PCT", "FT_PCT", 
                                                    "OREB", "DREB", "REB", "AST", "STL", "BLK", "TO", "PF", "PTS", "PLUS_MINUS"]].dropna()
datasets['game_details']['MIN'] = datasets['game_details']['MIN'].str.split(':').apply(lambda x: float(x[0]) + float(x[1])/60).round(3)
datasets['game_details'].head()

,GAME_ID,TEAM_ID,MIN,FG_PCT,FG3_PCT,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,22200477,1610612759,18.100,1.000,0.000,0.0,1.0,1.0,2.0,0.0,1.0,0.0,2.0,5.0,2.0,-2.0
1,22200477,1610612759,31.017,0.500,0.500,0.7,6.0,3.0,9.0,6.0,1.0,0.0,2.0,1.0,23.0,-14.0
2,22200477,1610612759,21.700,0.667,0.000,1.0,1.0,3.0,4.0,1.0,1.0,0.0,2.0,4.0,13.0,-4.0
3,22200477,1610612759,30.333,0.308,0.167,1.0,0.0,9.0,9.0,5.0,3.0,0.0,2.0,1.0,10.0,-18.0
4,22200477,1610612759,27.733,0.583,0.333,1.0,0.0,2.0,2.0,3.0,0.0,0.0,2.0,2.0,19.0,0.0


In [6]:
datasets['ranking'] = datasets['ranking'][["TEAM_ID", "SEASON_ID", "STANDINGSDATE", "CONFERENCE", "G", "W_PCT"]]
datasets['ranking']["STANDINGSDATE"] = pd.to_datetime(datasets['ranking']["STANDINGSDATE"])
datasets['ranking'].head(5)

,TEAM_ID,SEASON_ID,STANDINGSDATE,CONFERENCE,G,W_PCT
0,1610612743,22022,2022-12-22,West,30,0.633
1,1610612763,22022,2022-12-22,West,30,0.633
2,1610612740,22022,2022-12-22,West,31,0.613
3,1610612756,22022,2022-12-22,West,32,0.594
4,1610612746,22022,2022-12-22,West,33,0.576


In [7]:
datasets['games'] = datasets['games'].drop(['GAME_STATUS_TEXT', "TEAM_ID_home", "TEAM_ID_away"], axis=1)
datasets['games']["GAME_DATE_EST"] = pd.to_datetime(datasets['games']["GAME_DATE_EST"])
datasets['games'].head(5)

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2022-12-22,22200477,1610612740,1610612759,2022,126.0,0.484,0.926,0.382,25.0,46.0,117.0,0.478,0.815,0.321,23.0,44.0,1
1,2022-12-22,22200478,1610612762,1610612764,2022,120.0,0.488,0.952,0.457,16.0,40.0,112.0,0.561,0.765,0.333,20.0,37.0,1
2,2022-12-21,22200466,1610612739,1610612749,2022,114.0,0.482,0.786,0.313,22.0,37.0,106.0,0.470,0.682,0.433,20.0,46.0,1
3,2022-12-21,22200467,1610612755,1610612765,2022,113.0,0.441,0.909,0.297,27.0,49.0,93.0,0.392,0.735,0.261,15.0,46.0,1
4,2022-12-21,22200468,1610612737,1610612741,2022,108.0,0.429,1.000,0.378,22.0,47.0,110.0,0.500,0.773,0.292,20.0,47.0,0


#### Combining Information

In [8]:
datasets['games']

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2022-12-22,22200477,1610612740,1610612759,2022,126.0,0.484,0.926,0.382,25.0,46.0,117.0,0.478,0.815,0.321,23.0,44.0,1
1,2022-12-22,22200478,1610612762,1610612764,2022,120.0,0.488,0.952,0.457,16.0,40.0,112.0,0.561,0.765,0.333,20.0,37.0,1
2,2022-12-21,22200466,1610612739,1610612749,2022,114.0,0.482,0.786,0.313,22.0,37.0,106.0,0.470,0.682,0.433,20.0,46.0,1
3,2022-12-21,22200467,1610612755,1610612765,2022,113.0,0.441,0.909,0.297,27.0,49.0,93.0,0.392,0.735,0.261,15.0,46.0,1
4,2022-12-21,22200468,1610612737,1610612741,2022,108.0,0.429,1.000,0.378,22.0,47.0,110.0,0.500,0.773,0.292,20.0,47.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26646,2014-10-06,11400007,1610612737,1610612740,2014,93.0,0.419,0.821,0.421,24.0,50.0,87.0,0.366,0.643,0.375,17.0,43.0,1
26647,2014-10-06,11400004,1610612741,1610612764,2014,81.0,0.338,0.719,0.381,18.0,40.0,85.0,0.411,0.636,0.267,17.0,47.0,0
26648,2014-10-06,11400005,1610612747,1610612743,2014,98.0,0.448,0.682,0.500,29.0,45.0,95.0,0.387,0.659,0.500,19.0,43.0,1
26649,2014-10-05,11400002,1610612761,1610612758,2014,99.0,0.440,0.771,0.333,21.0,30.0,94.0,0.469,0.725,0.385,18.0,45.0,1


In [9]:
# aggregating the win percentages by team on that date
# first joining for the home team
datasets['games'] = datasets['games'].merge(datasets['ranking'], left_on=['HOME_TEAM_ID', 'GAME_DATE_EST'], right_on=['TEAM_ID', 'STANDINGSDATE'], how='left', suffixes=('', '_home_ranking')).rename(columns={'W_PCT': 'HOME_TEAM_W_PCT', 'CONFERENCE': 'HOME_TEAM_CONFERENCE'}).drop(['TEAM_ID', 'STANDINGSDATE', 'G'], axis=1)
# now we do the same for the away team
datasets['games'] = datasets['games'].merge(datasets['ranking'], left_on=['VISITOR_TEAM_ID', 'GAME_DATE_EST'], right_on=['TEAM_ID', 'STANDINGSDATE'], how='left', suffixes=('', '_away_ranking')).rename(columns={'W_PCT': 'VISITOR_TEAM_W_PCT', 'CONFERENCE': 'VISITOR_TEAM_CONFERENCE'}).drop(['TEAM_ID', 'STANDINGSDATE', 'G', 'SEASON_ID_away_ranking', 'SEASON_ID'], axis=1)
datasets['games'].keys()

Index(['GAME_DATE_EST', 'GAME_ID', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID', 'SEASON',
       'PTS_home', 'FG_PCT_home', 'FT_PCT_home', 'FG3_PCT_home', 'AST_home',
       'REB_home', 'PTS_away', 'FG_PCT_away', 'FT_PCT_away', 'FG3_PCT_away',
       'AST_away', 'REB_away', 'HOME_TEAM_WINS', 'HOME_TEAM_CONFERENCE',
       'HOME_TEAM_W_PCT', 'VISITOR_TEAM_CONFERENCE', 'VISITOR_TEAM_W_PCT'],
      dtype='object')

In [10]:
#fixing datatypes
datasets['games']['SEASON'] = datasets['games']['SEASON'].astype('str')

In [11]:
# now we check what data type we have for each column
datasets['games'].dtypes

GAME_DATE_EST              datetime64[ns]
GAME_ID                             int64
HOME_TEAM_ID                        int64
VISITOR_TEAM_ID                     int64
SEASON                             object
PTS_home                          float64
FG_PCT_home                       float64
FT_PCT_home                       float64
FG3_PCT_home                      float64
AST_home                          float64
REB_home                          float64
PTS_away                          float64
FG_PCT_away                       float64
FT_PCT_away                       float64
FG3_PCT_away                      float64
AST_away                          float64
REB_away                          float64
HOME_TEAM_WINS                      int64
HOME_TEAM_CONFERENCE               object
HOME_TEAM_W_PCT                   float64
VISITOR_TEAM_CONFERENCE            object
VISITOR_TEAM_W_PCT                float64
dtype: object

In [12]:
# nan values only for numerical variables, so we can fill them in with average
datasets['games'].fillna(datasets['games'].select_dtypes(include=['float64', 'int64']).mean(), inplace=True)

In [14]:
# now we have compiled a comprehensive dataset that we can do feature engineering on to make our models
# taking out the identifiers: game_id, home_id, visitor_team_id and copying the dataset
df = datasets['games'].copy().drop(['GAME_ID', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID', 'GAME_DATE_EST'], axis=1)
target = df.pop('HOME_TEAM_WINS')
df


,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_CONFERENCE,HOME_TEAM_W_PCT,VISITOR_TEAM_CONFERENCE,VISITOR_TEAM_W_PCT
0,2022,126.0,0.484,0.926,0.382,25.0,46.0,117.0,0.478,0.815,0.321,23.0,44.0,West,0.613,West,0.323
1,2022,120.0,0.488,0.952,0.457,16.0,40.0,112.0,0.561,0.765,0.333,20.0,37.0,West,0.543,East,0.364
2,2022,114.0,0.482,0.786,0.313,22.0,37.0,106.0,0.470,0.682,0.433,20.0,46.0,East,0.667,East,0.710
3,2022,113.0,0.441,0.909,0.297,27.0,49.0,93.0,0.392,0.735,0.261,15.0,46.0,East,0.600,East,0.235
4,2022,108.0,0.429,1.000,0.378,22.0,47.0,110.0,0.500,0.773,0.292,20.0,47.0,East,0.500,East,0.419
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26666,2014,93.0,0.419,0.821,0.421,24.0,50.0,87.0,0.366,0.643,0.375,17.0,43.0,East,1.000,West,0.500
26667,2014,81.0,0.338,0.719,0.381,18.0,40.0,85.0,0.411,0.636,0.267,17.0,47.0,East,0.000,East,1.000
26668,2014,98.0,0.448,0.682,0.500,29.0,45.0,95.0,0.387,0.659,0.500,19.0,43.0,West,1.000,West,0.000
26669,2014,99.0,0.440,0.771,0.333,21.0,30.0,94.0,0.469,0.725,0.385,18.0,45.0,East,1.000,West,0.000


In [15]:
# normalizing and ohe the data
categorical_columns = ['SEASON', 'HOME_TEAM_CONFERENCE', 'VISITOR_TEAM_CONFERENCE']
numerical_columns = ['PTS_home', 'FG_PCT_home', 'FT_PCT_home', 'FG3_PCT_home', 'AST_home', 'REB_home', 'PTS_away', 'FG_PCT_away', 'FT_PCT_away', 'FG3_PCT_away', 'AST_away', 'REB_away', 'HOME_TEAM_W_PCT', 'VISITOR_TEAM_W_PCT']

scaler = StandardScaler()
ohe = OneHotEncoder(handle_unknown='ignore')

transformer = ColumnTransformer([('scaler', scaler, numerical_columns), ('ohe', ohe, categorical_columns)], remainder='passthrough')

transformed_df = transformer.fit_transform(df)

ohe_columns = transformer.named_transformers_['ohe'].get_feature_names_out(input_features=categorical_columns)
all_columns = numerical_columns + list(ohe_columns)

# Convert the numpy array to a DataFrame
df_transformed = pd.DataFrame(transformed_df, columns=all_columns)

# adding back the target variable to the dataset
df_transformed['HOME_TEAM_WINS'] = target

df_transformed.head()

,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_W_PCT,VISITOR_TEAM_W_PCT,SEASON_2003,SEASON_2004,SEASON_2005,SEASON_2006,SEASON_2007,SEASON_2008,SEASON_2009,SEASON_2010,SEASON_2011,SEASON_2012,SEASON_2013,SEASON_2014,SEASON_2015,SEASON_2016,SEASON_2017,SEASON_2018,SEASON_2019,SEASON_2020,SEASON_2021,SEASON_2022,HOME_TEAM_CONFERENCE_East,HOME_TEAM_CONFERENCE_West,VISITOR_TEAM_CONFERENCE_East,VISITOR_TEAM_CONFERENCE_West,HOME_TEAM_WINS
0,1.699190,0.410891,1.648311,0.233954,0.419333,0.396775,1.218498,0.509733,0.544237,-0.260829,0.291614,0.289059,0.514020,-0.904342,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1
1,1.246794,0.481598,1.907018,0.909851,-1.316714,-0.510380,0.845829,2.006860,0.059811,-0.150969,-0.290883,-0.784524,0.160757,-0.697327,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1
2,0.794397,0.375537,0.255273,-0.387870,-0.159349,-0.963957,0.398626,0.365432,-0.744336,0.764533,-0.290883,0.595796,0.786538,1.049680,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1
3,0.718998,-0.349213,1.479156,-0.532062,0.805122,0.850352,-0.570313,-1.041506,-0.230844,-0.810130,-1.261712,0.595796,0.448414,-1.348668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1
4,0.342001,-0.561336,2.384631,0.197907,-0.159349,0.547967,0.696761,0.906562,0.137319,-0.526325,-0.290883,0.749165,-0.056248,-0.419624,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0


In [16]:
# checking for correlation between the variables and the target variable
df_transformed.corr()['HOME_TEAM_WINS'].sort_values(ascending=False)

HOME_TEAM_WINS                  1.000000
FG_PCT_home                     0.431611
HOME_TEAM_W_PCT                 0.396118
PTS_home                        0.394714
AST_home                        0.301019
FG3_PCT_home                    0.300957
REB_home                        0.245151
FT_PCT_home                     0.092896
HOME_TEAM_CONFERENCE_West       0.041981
VISITOR_TEAM_CONFERENCE_East    0.035941
SEASON_2012                     0.013025
SEASON_2010                     0.012279
SEASON_2007                     0.011459
SEASON_2005                     0.008566
SEASON_2004                     0.008457
SEASON_2009                     0.007131
SEASON_2008                     0.006930
SEASON_2015                     0.003311
SEASON_2011                     0.003022
SEASON_2006                     0.002031
SEASON_2022                     0.000444
SEASON_2018                     0.000363
SEASON_2017                    -0.002162
SEASON_2016                    -0.002998
SEASON_2003     

In [18]:
df_transformed.corr()

,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_W_PCT,VISITOR_TEAM_W_PCT,SEASON_2003,SEASON_2004,SEASON_2005,SEASON_2006,SEASON_2007,SEASON_2008,SEASON_2009,SEASON_2010,SEASON_2011,SEASON_2012,SEASON_2013,SEASON_2014,SEASON_2015,SEASON_2016,SEASON_2017,SEASON_2018,SEASON_2019,SEASON_2020,SEASON_2021,SEASON_2022,HOME_TEAM_CONFERENCE_East,HOME_TEAM_CONFERENCE_West,VISITOR_TEAM_CONFERENCE_East,VISITOR_TEAM_CONFERENCE_West,HOME_TEAM_WINS
PTS_home,1.000000,0.662338,0.186596,0.422753,0.605510,0.162789,0.487787,0.131725,0.071337,0.056667,0.215549,-0.140480,0.180534,-0.171497,-0.140200,-0.084992,-0.090836,-0.064612,-0.039037,-0.045524,-0.032405,-0.048446,-0.095135,-0.072995,-0.030996,-0.042325,0.004435,0.061375,0.066201,0.155537,0.146753,0.150717,0.136681,0.117499,-0.104597,0.104597,-0.065231,0.065231,0.394714
FG_PCT_home,0.662338,1.000000,0.020835,0.452027,0.550701,-0.188009,0.118373,0.041729,0.029857,0.014234,0.015947,-0.465553,0.183178,-0.177866,-0.060106,-0.025110,-0.002807,0.005525,0.010340,0.012374,0.021891,0.013009,-0.026172,-0.008492,-0.005440,-0.031806,-0.019834,0.010828,0.018548,0.016193,0.008932,0.025625,0.015147,0.031376,-0.069341,0.069341,-0.024022,0.024022,0.431611
FT_PCT_home,0.186596,0.020835,1.000000,0.029065,0.022136,-0.051648,0.077959,0.048291,0.036496,0.021644,0.032207,-0.114624,0.046046,-0.016020,-0.025306,-0.009496,-0.037058,-0.027816,-0.007483,0.014191,-0.006312,0.008587,-0.014168,-0.026461,-0.000463,-0.024786,-0.000668,0.023227,0.013829,0.015131,0.020688,0.037282,0.033392,0.025068,-0.017762,0.017762,-0.008788,0.008788,0.092896
FG3_PCT_home,0.422753,0.452027,0.029065,1.000000,0.352908,-0.088836,0.045707,0.007957,0.014097,-0.002935,-0.003069,-0.204562,0.112271,-0.103192,-0.025550,-0.002236,-0.012830,-0.006094,0.008236,0.016596,-0.013448,-0.000363,-0.021583,0.015844,0.000274,-0.013459,-0.003162,0.018484,0.010315,0.001582,0.007450,0.021265,-0.002340,0.000123,-0.020738,0.020738,-0.009744,0.009744,0.300957
AST_home,0.605510,0.550701,0.022136,0.352908,1.000000,0.066785,0.191105,0.008973,0.028282,-0.006505,0.146240,-0.108799,0.135982,-0.166462,-0.032248,-0.040153,-0.076908,-0.037548,-0.015012,-0.056736,-0.043120,-0.038911,-0.057947,-0.010085,-0.022331,-0.014454,-0.009153,0.021022,0.036457,0.096146,0.080307,0.086081,0.096857,0.068186,-0.052583,0.052583,-0.011751,0.011751,0.301019
REB_home,0.162789,-0.188009,-0.051648,-0.088836,0.066785,1.000000,-0.151172,-0.474220,-0.116333,-0.206857,-0.097878,0.067585,0.098314,-0.107321,-0.006646,-0.029555,-0.056761,-0.063298,-0.024061,-0.050852,-0.042009,-0.052641,-0.016186,-0.022206,-0.004235,0.018021,0.035093,0.023984,0.026209,0.088489,0.068892,0.049910,0.048021,0.026692,-0.032355,0.032355,0.003265,-0.003265,0.245151
PTS_away,0.487787,0.118373,0.077959,0.045707,0.191105,-0.151172,1.000000,0.668828,0.204778,0.409356,0.595925,0.176331,-0.192820,0.169086,-0.154828,-0.091119,-0.097277,-0.065504,-0.054652,-0.048659,-0.033738,-0.057788,-0.096373,-0.080203,-0.024638,-0.032436,0.001996,0.055434,0.074464,0.155071,0.161603,0.177709,0.149632,0.113641,-0.049316,0.049316,-0.095489,0.095489,-0.413341
FG_PCT_away,0.131725,0.041729,0.048291,0.007957,0.008973,-0.474220,0.668828,1.000000,0.040512,0.437082,0.525501,-0.170627,-0.187831,0.167388,-0.072516,-0.033738,-0.007260,0.004180,-0.005895,0.008082,0.027178,0.003010,-0.031010,-0.019474,-0.006476,-0.022710,-0.018507,0.004010,0.022436,0.019571,0.024569,0.054168,0.030616,0.031705,-0.015716,0.015716,-0.055752,0.055752,-0.434892
FT_PCT_away,0.071337,0.029857,0.036496,0.014097,0.028282,-0.116333,0.204778,0.040512,1.000000,0.032350,0.031284,-0.036675,-0.046400,0.052843,-0.016499,-0.014713,-0.036639,-0.024579,-0.014807,0.018841,-0.008873,0.002150,-0.013839,-0.009073,-0.020024,-0.028314,-0.009221,0.024624,0.017451,0.018502,0.026348,0.035776,0.038207,0.027393,0.004558,-0.004558,-0.037889,0.037889,-0.108325
FG3_PCT_away,0.056667,0.014234,0.021644,-0.002935,-0.006505,-0.2